# Introduction to Data Consistency 

## Table of Contents
* [01. Importing Libraries](#01.-Importing-Libraries)
* [02. Importing CSV Files](#02.-Importing-CSV-Files)
* [03. Data Consistency Check for Product File](#03.-Data-Consistency-Check-for-Product-File)
* [04. Data Consistency Check for Order File](#04.-Data-Consistency-Check-for-Order-File)
* [05. Exporting Files](#05.-Exporting-Files)

# 01. Importing Libraries

In [1]:
# Importing the libraries needed
import pandas as pd
import numpy as np
import os

# 02. Importing CSV Files

In [2]:
# Importing Product CSV File
df_prods = pd.read_csv(r'/Users/suzandiab/Documents/Instacart Basket Analysis/02 Data/Original Data/products.csv', 
                      index_col = False)

In [3]:
# Importing manipulated order CSV File
df_ords = pd.read_csv(r'/Users/suzandiab/Documents/Instacart Basket Analysis/02 Data/Prepared Data/orders_wrangled.csv', 
                      index_col = 0)

In [4]:
# Create test data frame for mixed-type
df_test = pd.DataFrame()

In [5]:
# Create mixed type column
df_test['mix'] = ['a', 'b', 1, True]

In [6]:
# Check test data frame
df_test.head()

,mix
0,a
1,b
2,1
3,True


In [7]:
# Checking for mixed type 
for col in df_test.columns.tolist():
  weird = (df_test[[col]].map(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_test[weird]) > 0:
    print (col)

mix


In [8]:
# Changing data type of mix column to string
df_test['mix'] = df_test['mix'].astype('str')

# 03. Data Consistency Check for Product File

In [9]:
# Checking for missing values in product file
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [10]:
# Creating subset for missing values
df_nan = df_prods[df_prods['product_name'].isnull() == True]

In [11]:
# Viewing missing values subset
df_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


In [12]:
# Checking rows of product file
df_prods.shape

(49693, 5)

In [13]:
# Creating new subset data frame with only non-missing values
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]

In [14]:
# Checking number of rows of new data frame
df_prods_clean.shape

(49677, 5)

In [15]:
# Looking for duplicates in clean data frame
df_dups = df_prods_clean[df_prods_clean.duplicated()]

In [16]:
# Viewing duplicates
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [17]:
# New data frame without duplicates
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [18]:
# Checking number of rows
df_prods_clean_no_dups.shape

(49672, 5)

In [19]:
# Checking descriptivess for prods data set
df_prods_clean_no_dups.describe()

,product_id,aisle_id,department_id,prices
count,49672.000000,49672.000000,49672.000000,49672.000000
mean,24850.349775,67.762442,11.728942,9.993282
std,14340.705287,38.315784,5.850779,453.615536
min,1.000000,1.000000,1.000000,1.000000
25%,12432.750000,35.000000,7.000000,4.100000
50%,24850.500000,69.000000,13.000000,7.100000
75%,37268.250000,100.000000,17.000000,11.100000
max,49688.000000,134.000000,21.000000,99999.000000


Maximum price is a lot higher than the mean

In [20]:
# Identifying high prices 
df_prods_clean_no_dups[df_prods_clean_no_dups["prices"]> 50.0]

,product_id,product_name,aisle_id,department_id,prices
21554,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0
33666,33664,2 % Reduced Fat Milk,84,16,99999.0


In [21]:
# Replacing these abnormal values with more normal values
df_prods_clean_no_dups = df_prods_clean_no_dups.replace({"prices":{99999.0: 9.99, 14900.0:1.49}})

In [22]:
# Checking for high values
df_prods_clean_no_dups[df_prods_clean_no_dups["prices"]> 50.0]

,product_id,product_name,aisle_id,department_id,prices


# 04. Data Consistency Check for Order File

In [23]:
# Descriptive stats for order file
df_ords.describe()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


Nothing looks off or needs to be investigated further.

In [24]:
# Checking for mixed type columns
for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].map(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col)

In [25]:
# Checking for missing values in order file
df_ords.isnull().sum()

order_id                      0
user_id                       0
order_number                  0
orders_day_of_week            0
order_hour_of_day             0
days_since_last_order    206209
dtype: int64

There are 206209 missing values in days_since_last_order column. This could be because the person didn't re-order.

In [26]:
# Checking number of columns before filtering out missing values
df_ords.shape

(3421083, 6)

In [27]:
# Checking for duplicates
df_dups = df_ords[df_ords.duplicated()]

In [28]:
# Checking df for duplicate values
df_dups

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order


There are no duplicates.

# 05. Exporting Files

In [29]:
# Exporting clean order CSV file
df_ords.to_csv(r'/Users/suzandiab/Documents/Instacart Basket Analysis/02 Data/Prepared Data/orders_checked.csv')

In [30]:
# Exporting clean product CSV file
df_prods_clean_no_dups.to_csv(r'/Users/suzandiab/Documents/Instacart Basket Analysis/02 Data/Prepared Data/products_checked.csv')